<h1 align=center><font size = 6>Segmenting and Clustering Neighborhoods in New York City</font></h1>

### Install Beautifulsoup Library

In [3]:
!conda install -c anaconda beautifulsoup4 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    beautifulsoup4-4.8.2       |           py36_0         161 KB  anaconda
    ca-certificates-2019.11.27 |                0         132 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.2-py36_0      anaconda
    ca-certificates: 2019.11.27-0      --> 2019.11.27-0      anaconda
    certifi:         2019.11.28-py36_0 --> 2019.11.28-py36_0 anaconda
    openssl:         1.1.1d-

### Import BeautifulSoup and Requests

In [32]:
from bs4 import BeautifulSoup
import requests
import lxml.html as lh

### Parse the html page

In [67]:
html_path = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(html_path)
doc = lh.fromstring(page.content)
html_page = page.text

tr_elements = doc.xpath('//tr')

AttributeError: 'list' object has no attribute 'text_content'

### Parse Table Header

In [71]:
col = []
i = 0
for t in tr_elements[0]:
    i += 1
    name = t.text_content().rstrip('\r\n') #strip the trailing new line
    print(name)
    print('%d: "%s"'%(i, name))
    col.append((name,[]))

Postcode
1: "Postcode"
Borough
2: "Borough"
Neighborhood
3: "Neighborhood"


### Create Pandas DataFrame

In [115]:
for j in range(1,len(tr_elements)):
    
    T = tr_elements[j] #jth row
    
    if len(T) != 3: #check if it is the lenght of our header
        break
    
    i = 0 # index of the column
    
    for t in T.iterchildren(): #iterate to each element of the row
        data = t.text_content().rstrip('\r\n') #strip the trailing newline
        
        if i > 0: #check if row is empty
            try:
                data = int(data)
            except:
                pass
            
        col[i][1].append(data) #append data to the empty list of the i'th column
        i += 1
        

In [116]:
dic = {title:column for (title, column) in col}
df =  pd.DataFrame(dic)
print(df.head())
print("\nDataFrame shape is: {}".format(df.shape))

  Postcode           Borough      Neighborhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

DataFrame shape is: (861, 3)


#### Copy value of Borough to Neighborhood if Borough is != "Not assigned"

In [117]:
df.Neighborhood = np.where(df.Neighborhood == 'Not assigned', df.Borough, df.Neighborhood) 
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [118]:
df.drop(df[df['Neighborhood'] == 'Not assigned'].index)
df.reset_index(drop = True, inplace = True) #reset index
df.head(15)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Queen's Park


In [114]:
print("New DataFrame shape is: {}".format(df.shape))

New DataFrame shape is: (574, 3)
